 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
Tangi Migot et Paul Raynaud

In [1]:
using Pkg
Pkg.activate(".") #Accède au fichier Project.toml
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Desktop/MTH8408/Git/MTH8408-Hiv24/lab2`


Precompiling 

project...


  ✓ Plots


  ✓ Plots → FileIOExt
  ✓ Plots → IJuliaExt


  ✓ Plots → UnitfulExt


  ✓ MathOptInterface


  ✓ JuMP


  ✓ OptimizationProblems
  7 dependencies successfully precompiled in 308 seconds. 199 already precompiled.


Status `~/Desktop/MTH8408/Git/MTH8408-Hiv24/lab2/Project.toml`
  [54578032] ADNLPModels v0.7.0
  [7073ff75] IJulia v1.24.2
  [40e66cde] LDLFactorizations v0.10.1
  [a4795742] NLPModels v0.20.0
  [5049e819] OptimizationProblems v0.7.3
⌅ [91a5bcdd] Plots v1.39.0
  [37e2e46d] LinearAlgebra
  [de0858da] Printf
  [8dfed614] Test
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [2]:
using ADNLPModels, LinearAlgebra, NLPModels, Printf

On pourra trouver de la documentation sur `ADNLPModels` et `NLPModels` ici:
- [juliasmoothoptimizers.github.io/NLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/NLPModels.jl/dev/)
- [juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/)

In [3]:
# Problème test:
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
g(x) = 6 * [x[1]^2 - x[1] - 2*x[1]*x[2] + x[2]^2 + x[2]; -x[1]^2 + 2*x[1]*x[2] + x[1]] # le gradient de f
H(x) = 6 * [2*x[1]-1-2*x[2] -2*x[1]+2*x[2]+1; -2*x[1]+2*x[2]+1 2*x[1]] # la Hessienne de f

H (generic function with 1 method)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

Ci-dessous, vous avez le code de deux fonctions qui ont été vues dans le cours, la recherche linéaire qui satisfait Armijo, et une méthode de Newton avec cette recherche linéaire. Le but de ce laboratoire est d'implémenter d'autres méthodes utiles pour résoudre des problèmes de grandes dimensions.

In [4]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end

armijo (generic function with 1 method)

In [5]:
#Test pour vérifier que la fonction armijo fonctionne correctement.
using Test #le package Test définit (entre autre) la macro @test qui permet de faire des tests unitaires :-)
xk = ones(2)
gk = g(xk)
dk = - gk
fk = f(xk)
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= fk + 1.0e-4 * t * dot(gk,dk)

xk = [1.5, 0.5]
fk = f(xk)
gk = g(xk)
dk = - gk
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= f(xk) + 1.0e-4 * t * dot(g(xk),dk)


Test Passed

In [123]:
function newton_armijo(f, g, H, x0; verbose::Bool = true)
  xk  = x0
  fk  = f(xk)
  gk = g(xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    Hk = H(xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
    α = armijo(xk, dk, fk, gk, f)
    xk += α * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm α
  end
  return xk
end

newton_armijo (generic function with 2 methods)

In [7]:
sol = newton_armijo(f, g, H, [.5, .5])
@test g(sol) ≈ zeros(2) atol = 1.0e-6

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00


 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


Test Passed

On veut améliorer le code de la fonction `newton_armijo` avec les ajouts suivants:
- Changer les paramètre d'entrées de la fonction pour un `nlp`
- Avant d'appeler la recherche linéaire, si `slope = dot(dk, gk)` est plus grand que `-1.0e-4 * norm(dk) * gnorm`, on modifie le système. On fait maximum 5 mise à jour de `λ`, sinon on prend l'opposé du gradient.
```
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
```
Ajouter un compteur sur le nombre de mises à jour de `λ` et ajuster `dk = - gk` si la limite est atteinte.
- On veut aussi détecter et éventuellement arrêter la boucle `while` si la fonction objectif `fk` devient trop petite/négative (inférieure à `-1e15`), i.e. le problème est non-bornée inférieurement.
- On veut ajouter deux critères d'arrêts supplémentaires: 
  - un compteur sur le nombre d'évaluations de f (maximum 1000). Utiliser `neval_obj(nlp)`.
  -  une limite de temps d'execution, `max_time = 60.0`. Utiliser la fonction `time()`.
- Enfin, on voudrait aussi voir un message à l'écran si l'algorithme n'a pas trouvé la solution, i.e. il s'est arrêté à cause de la limite sur le nombre d'itérations, temps, évaluation de fonctions, problème non-borné.

In [9]:
x0 = [0.0, 0.0]
nlp = ADNLPModel(f, x0)

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  ForwardDiffADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                            

In [122]:
function newton_armijo(nlp, x0; verbose::Bool = true)
  t0 = time()
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = hess(nlp, xk)
    if neval_obj(nlp) > 1000
      println("Budget d'évaluations atteint")
      break
    end
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    nb_maj = 0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      if nb_maj <= 5
        λ = max(1.0e-3, 10 * λ)
        dk = - ((Hk + λ * I ) \ gk)
        slope = dot(dk, gk)
      else
        dk = - gk
        break
      end
      nb_maj +=1
    end
    α = armijo(xk, dk, fk, gk, f)
    xk += α * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm α
    t1 = time()
    if t1 - t0 > 60
      println("Temps maximal d'éxécution atteint")
      break
    end
  end
  if fk <= -1e15
    println("Le problème est non-borné")
  end
  if k == 100
    println("Nombre maximal d'itérations atteint")
  end
  return xk
end

newton_armijo (generic function with 2 methods)

In [106]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0 = [0.5, 0.5]
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366509693e-8
 2.3230573665124037e-8

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$
Ce type de factorisation n'est possible que si la matrice hessienne est définie positive, dans le cas contraire on a besoin de régularisé le système comme dans l'exercice précédent.

Pour résoudre le système linéaire en utilisant cette factorisation, on va utiliser le package [`LDLFactorizations`](https://github.com/JuliaSmoothOptimizers/LDLFactorizations.jl):

In [19]:
using LDLFactorizations, LinearAlgebra

Un tutoriel sur l'utilisation de `LDLFactorizations` est disponible sur la documentation du package sur github ou encore [à ce lien](https://juliasmoothoptimizers.github.io/LDLFactorizations.jl/dev/tutorial/).

Voici un exemple d'utilisation de ce package. La matrice dont on veut calculer la factorisation doit être de type `Symmetric`.

In [20]:
A = ones(2,2) #cette matrice symétrique, mais pas du type Symmetric
              #à noter que cette matrice n'est pas définie positive.
typeof(A) <: Symmetric #false
A = Symmetric(A)
typeof(A) <: Symmetric #true :)

true

Deuxième étape, le package fait une phase d'analyse de la matrice avec `ldl_analyze` en créant une structure pratique pour les diverses fonctions du package.

In [21]:
A = -rand(2, 2)
sol = rand(2)
b = A*sol #on veut résoudre le système A*x=b

# LDLFactorizations va en réalité demander la matrice triangulaire supérieure
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)
x = S \ b # x = A \b ça va être résolu par Julia 
norm(A * x - b)

0.0

In [22]:
A = [0. 1.; 1. 0.]

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [23]:
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)

LDLFactorizations.LDLFactorization{Float64, Int64, Int64, Int64}(true, false, true, 2, [2, -1], [0, 0], [1, 2], [1, 2], [1, 2], [1, 2, 2], [1, 1, 1], Int64[], [5010176848], [2.1977934847e-314], [0.0, 2.2984721425e-314], [0.0, 2.2984721425e-314], [4652174368, 5015892960], 0.0, 0.0, 0.0, 2)

In [24]:
S.d

2-element Vector{Float64}:
 0.0
 2.2984721425e-314

La matrice `A` factorisée par $LDL^T$ n'était pas forcément définie positive. On peut le voir sur les valeurs de $D$.

In [25]:
S.d #c'est le vecteur qui correspond à la matrice diagonale D.

2-element Vector{Float64}:
 0.0
 2.2984721425e-314

Pour l'optimisation, dans le cas où des valeurs de $D$ sont négatives, i.e. `minimum(S.d) <= 0.`, on ajoutera une correction pour être sûr d'obtenir une direction de descente. On pourra choisir un des deux:
- `S.d   = abs.(S.d)`
- `S.d .+= -minimum(S.d) + 1e-6`

#### Utiliser cette technique pour calculer la direction de descente:

In [120]:
function newton_ldlt_armijo(nlp, x0; verbose::Bool = true)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = Symmetric(triu(hess(nlp, xk)), :U)
    Sk = ldl_analyze(Hk)
    ldl_factorize!(Hk, Sk)
    dk = - Sk \ gk
    α = armijo(xk, dk, fk, gk, x -> obj(nlp, x))
    xk += α * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm α
  end
  return xk
end

newton_ldlt_armijo (generic function with 1 method)

In [43]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))

newton_ldlt_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366509693e-8
 2.3230573665124037e-8

### Exercice 3: Méthode quasi-Newton: BFGS

#### Méthode quasi-Newton: BFGS
Pour des problèmes de très grandes tailles, il est parfois très coûteux d'évaluer la hessienne du problème d'optimisation (et même le produit hessienne-vecteur). La famille des méthode *quasi-Newton* construit une approximation $B_k$ symétrique de la matrice Hessienne en utilisant seulement le gradient et en mesurant sa variation, et permet quand même d'améliorer significativement les performances comparé à la méthode du gradient.
$$
s_k = x_{k+1} - x_k, \quad y_k = \nabla f(x_{k+1}) - \nabla f(x_k).
$$
Par ailleurs la matrice $B_k$ est aussi construite de façon à ce que l'inverse soit connue, il n'y a donc pas de système linéaire à résoudre.

La méthode la plus connue dans la famille des méthodes quasi-Newton, est la méthode BFGS (Broyden - Fletcher, Goldfarb, and Shanno) où $B_k$ est définir positive ($B_0 = \lambda I, \;\lambda > 0$).
La formule suivante calcule l'inverse de $B_k$ que l'on note $H_k$:
$$
H_{k+1} = (I - \rho_k s_ky_k^T)H_k(I-\rho_ky_ks_k^T) + \rho_ks_ks_k^T, \quad \rho_k = \frac{1}{y_k^Ts_k}.
$$
L'algorithme est presque le même que la méthode de Newton à la différence qu'il n'y a pas de système linéaire à résoudre et la direction $d_k$ est à coup sûr une direction de descente. Ainsi la direction de descente est calculée comme suit:
$$
d_k = - H_k \nabla f(x_k).
$$

Comment choisir la matrice $H_0$? On peut éventuellement choisir $I$. Une alternative est d'utiliser $H_0=I$ pour la première itération et ensuite mettre $H_0$ à jour avant de calculer $H_1$ en utilisant:
$$
H_0 = \frac{y_k^Ts_k}{y_k^Ty_k}I.
$$

**Important**: pour s'assurer que la matrice $H_k$ reste définie positive à toutes les itérations, il faut s'assurer que $y_k^Ts_k>0$. C'est toujours vrai pour des fonctions convexes, mais pas nécessairement dans le cas général. On pourra tester ici la version "skip" qui ne mets pas à jour quand cette condition n'est pas vérifiée.

In [124]:
function bfgs_quasi_newton_armijo(nlp, x0; verbose::Bool = true)
  t0 = time()
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  Hk = I(length(xk))
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    dk = -Hk * gk
    α = armijo(xk, dk, fk, gk, f)  
    sk = α*dk
    xk += sk
    new_fk = obj(nlp, xk)
    new_gk = grad(nlp, xk)
    yk = new_gk - gk
    if yk' * sk > 0
      ρk = 1 / dot(yk, sk)
      Hk = (I - ρk * sk * yk') * Hk * (I - ρk * yk * sk') + ρk * sk * sk'
    end
    fk, gk = new_fk, new_gk
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm α
    t1 = time()
    if t1 - t0 > 60
      println("Temps maximal d'éxécution atteint")
      break
    end
  end
  if fk <= -1e15
    println("Le problème est non-borné")
  end
  if k == 100
    println("Nombre maximal d'itérations atteint")
  end
  return xk
end

bfgs_quasi_newton_armijo (generic function with 1 method)

In [108]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, x0)

bfgs_quasi_newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  3.33e-01   4.4e+00 3.0e-01 
 2  2.02e-01   1.5e+00 4.4e-01 
 3 -3.77e-01   1.1e+00 1.0e+00 
 4 -7.45e-01   3.0e+00 5.1e-03 
 5 -8.57e-01   1.7e+00 1.0e+00 
 6 -9.03e-01   1.6e+00 1.0e+00 
 7 -9.64e-01   1.0e+00 1.0e+00 
 8 -9.88e-01   5.5e-01 1.0e+00 
 9 -1.00e+00   5.3e-02 1.0e+00 
10 -1.00e+00   1.6e-03 1.0e+00 
11 -1.00e+00   7.3e-05 1.0e+00 
12 -1.00e+00   3.0e-07 1.0e+00 


2-element Vector{Float64}:
 0.9999999947853835
 1.9103013141847714e-8

### Exercice 4: application à un problème de grande taille

On va ajouter le package `OptimizationProblems` qui contient, comme son nom l'indique, une collection de problème d'optimisation disponible au format de `JuMP` (dans le sous-module `OptimizationProblems.PureJuMP`) et de `ADNLPModel` (dans le sous-module `OptimizationProblems.ADNLPProblems`).

In [68]:
using ADNLPModels, OptimizationProblems.ADNLPProblems # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!

In [93]:
n = 500
model = genrose(n=n)
@test typeof(model) <: ADNLPModel

Test Passed

Si vous le souhaitez, il est possible d'accéder à certaines informations sur le problème en accédant à son meta:

In [70]:
using OptimizationProblems
OptimizationProblems.genrose_meta

Dict{Symbol, Any} with 17 entries:
  :has_equalities_only    => false
  :origin                 => :unknown
  :has_inequalities_only  => false
  :defined_everywhere     => missing
  :has_fixed_variables    => false
  :variable_ncon          => false
  :nvar                   => 100
  :is_feasible            => true
  :minimize               => true
  :ncon                   => 0
  :name                   => "genrose"
  :best_known_lower_bound => -Inf
  :objtype                => :other
  :best_known_upper_bound => 405.106
  :has_bounds             => false
  :variable_nvar          => true
  :contype                => :unconstrained

Il est aussi possible d'accéder au meta de tous les problèmes

In [71]:
OptimizationProblems.meta

Row,nvar,variable_nvar,ncon,variable_ncon,minimize,name,has_equalities_only,has_inequalities_only,has_bounds,has_fixed_variables,objtype,contype,best_known_lower_bound,best_known_upper_bound,is_feasible,defined_everywhere,origin
,Int64,Bool,Int64,Bool,Bool,String,Bool,Bool,Bool,Bool,Symbol,Symbol,Real,Real,Bool?,Bool?,Symbol
1,1,false,0,false,true,AMPGO02,false,false,false,false,other,unconstrained,-Inf,0.839498,true,missing,unknown
2,1,false,0,false,true,AMPGO03,false,false,false,false,other,unconstrained,-Inf,2.88961,true,missing,unknown
3,1,false,0,false,true,AMPGO04,false,false,false,false,other,unconstrained,-Inf,-2.5666,true,missing,unknown
4,1,false,0,false,true,AMPGO05,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown
5,1,false,0,false,true,AMPGO06,false,false,false,false,other,unconstrained,-Inf,3.5177e-43,true,missing,unknown
6,1,false,0,false,true,AMPGO07,false,false,false,false,other,unconstrained,-Inf,2.56475,true,missing,unknown
7,1,false,0,false,true,AMPGO08,false,false,false,false,other,unconstrained,-Inf,-2.0928,true,missing,unknown
8,1,false,0,false,true,AMPGO09,false,false,false,false,other,unconstrained,-Inf,0.921136,true,missing,unknown
9,1,false,0,false,true,AMPGO10,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown


Résoudre le problème `genrose` et un autre problème de la collection en utilisant vos algorithmes précédents.
Avant d'utiliser l'algorithme on testera que le problème est bien sans contrainte avec:

In [72]:
unconstrained(nlp) #qui retourne vrai si `nlp` est un problème sans contraintes.

true

In [111]:
x0 = rand(500)

newton_armijo(model, x0)

 k        fk ||∇f(x)||
 0  1.02e+04   3.4e+03


Budget d'évaluations atteint


500-element Vector{Float64}:
 0.9019032924146813
 0.7106945322386807
 0.8076744551109812
 0.3330387357785468
 0.04643216428146568
 0.49773855826683766
 0.22952224580756297
 0.7481061697631094
 0.8781221894532957
 0.1267553852584702
 ⋮
 0.18232985832530724
 0.6065978698288902
 0.7817999879291717
 0.37335503142553483
 0.4967981471071574
 0.49594113154009734
 0.829714144512861
 0.21319213320154373
 0.538085028503373

In [110]:
newton_ldlt_armijo(model, x0)

 k        fk ||∇f(x)||
 0  1.12e+04   3.6e+03


 1  1.09e+04   3.5e+03 8.8e-02 
 2  9.11e+03   3.5e+03 3.0e-01 


 3  8.65e+03   3.8e+03 3.0e-01 
 4  8.55e+03   3.7e+03 3.9e-02 


 5  7.99e+03   3.3e+03 1.3e-01 
 6  7.93e+03   3.3e+03 1.7e-02 


 7  7.56e+03   2.9e+03 2.0e-01 
 8  6.44e+03   2.5e+03 4.4e-01 


...

93  3.49e+03   8.4e+02 3.2e-19 
94  3.49e+03   8.4e+02 3.2e-19 


95  3.49e+03   8.4e+02 3.2e-19 
96  3.49e+03   8.4e+02 3.2e-19 


97  3.49e+03   8.4e+02 3.2e-19 
98  3.49e+03   8.4e+02 3.2e-19 


99  3.49e+03   8.4e+02 3.2e-19 
100  3.49e+03   8.4e+02 3.2e-19 


500-element Vector{Float64}:
 -0.03635808371619757
  0.07933243665039821
  0.19670530800113403
  0.1440453525046628
  0.023072649161883136
 -0.02530559877722868
 -0.6218737715602947
  0.843210249140701
  0.9767284706447285
  1.0763855125481492
  ⋮
 -0.11424321139824808
 -0.02207173165734683
  0.03282255984799271
  0.5208758308438609
  0.7732570592233209
  0.9030045219762872
  0.9451713478055382
  0.9049564449220165
  0.7518954861790245

In [112]:
bfgs_quasi_newton_armijo(model, x0)

 k        fk ||∇f(x)||
 0  1.02e+04   3.4e+03
 1  1.09e+14   1.6e+11 1.0e+00 
 2  8.60e+13   2.0e+11 1.0e+00 


 3  2.97e+27   4.2e+21 4.0e-05 
 4  1.07e+42   2.3e+32 3.0e-01 


 5  1.51e+64   1.1e+49 3.9e-02 
 6 6.24e+101   1.4e+77 3.5e-09 
 7 7.08e+247  7.9e+186 1.0e+00 
 8       Inf       NaN 1.0e+00 


500-element Vector{Float64}:
 -1.5567595002388858e169
 -5.425070985680966e168
 -1.5567595002388858e169
  1.0378396668259239e169
 -4.7078263015400106e57
  2.1511585821482786e170
  5.660943637232312e168
 -9.906651365156546e168
 -1.5095849699286165e169
  1.9624604609072015e170
  ⋮
  3.1385508676933404e57
 -2.506146922733055e167
  3.4437407126496565e169
 -2.8776463489264253e169
 -1.5331722350837512e168
 -1.2737123183772702e169
 -3.019169939857233e169
 -1.4492015711314719e171
  2.4766628412891365e168

In [113]:
n = 100
model2 = woods(n=n)
@test typeof(model) <: ADNLPModel

Test Passed

In [114]:
unconstrained(model2)

true

In [117]:
x0 = rand(100)

newton_armijo(model2, x0)

 k        fk ||∇f(x)||
 0  1.35e+03   8.8e+02
Budget d'évaluations atteint


100-element Vector{Float64}:
 0.025870851438840825
 0.7236800335600513
 0.8431026314835876
 0.05301929967410379
 0.6630100927407507
 0.6605173118490606
 0.0839009152720559
 0.5552043609461035
 0.35217434046854734
 0.5264521121433411
 ⋮
 0.8135468075678837
 0.055335244766780756
 0.8413990676619839
 0.19017656641736758
 0.91825201886436
 0.10784832727712335
 0.48872906033687
 0.408460933119133
 0.047637378587892854

In [116]:
newton_ldlt_armijo(model2, x0)

 k        fk ||∇f(x)||
 0  1.06e+03   7.2e+02
 1  1.05e+03   7.3e+02 7.7e-03 
 2  8.99e+02   8.8e+02 6.7e-01 
 3  3.33e+02   3.6e+02 1.0e+00 
 4  3.20e+02   3.0e+02 2.0e-01 


 5  3.20e+02   3.0e+02 1.9e-17 
 6  3.20e+02   3.0e+02 2.5e-18 
 7  3.20e+02   3.0e+02 7.3e-19 
 8  3.20e+02   3.0e+02 1.4e-19 
 9  3.20e+02   3.0e+02 1.4e-19 
10  3.20e+02   3.0e+02 1.4e-19 


11  3.20e+02   3.0e+02 1.4e-19 
12  3.20e+02   3.0e+02 1.4e-19 
13  3.20e+02   3.0e+02 1.4e-19 
14  3.20e+02   3.0e+02 1.4e-19 
15  3.20e+02   3.0e+02 1.4e-19 
16  3.20e+02   3.0e+02 1.4e-19 
17  3.20e+02   3.0e+02 1.4e-19 
18  3.20e+02   3.0e+02 1.4e-19 
19  3.20e+02   3.0e+02 1.4e-19 
20  3.20e+02   3.0e+02 1.4e-19 
21  3.20e+02   3.0e+02 1.4e-19 
22  3.20e+02   3.0e+02 1.4e-19 


23  3.20e+02   3.0e+02 1.4e-19 
24  3.20e+02   3.0e+02 1.4e-19 
25  3.20e+02   3.0e+02 1.4e-19 
26  3.20e+02   3.0e+02 1.4e-19 


27  3.20e+02   3.0e+02 1.4e-19 
28  3.20e+02   3.0e+02 1.4e-19 
29  3.20e+02   3.0e+02 1.4e-19 
30  3.20e+02   3.0e+02 1.4e-19 
31  3.20e+02   3.0e+02 1.4e-19 
32  3.20e+02   3.0e+02 1.4e-19 
33  3.20e+02   3.0e+02 1.4e-19 
34  3.20e+02   3.0e+02 1.4e-19 
35  3.20e+02   3.0e+02 1.4e-19 
36  3.20e+02   3.0e+02 1.4e-19 


...

61  3.20e+02   3.0e+02 1.4e-19 
62  3.20e+02   3.0e+02 1.4e-19 
63  3.20e+02   3.0e+02 1.4e-19 
64  3.20e+02   3.0e+02 1.4e-19 
65  3.20e+02   3.0e+02 1.4e-19 
66  3.20e+02   3.0e+02 1.4e-19 
67  3.20e+02   3.0e+02 1.4e-19 
68  3.20e+02   3.0e+02 1.4e-19 
69  3.20e+02   3.0e+02 1.4e-19 
70  3.20e+02   3.0e+02 1.4e-19 
71  3.20e+02   3.0e+02 1.4e-19 
72  3.20e+02   3.0e+02 1.4e-19 


73  3.20e+02   3.0e+02 1.4e-19 
74  3.20e+02   3.0e+02 1.4e-19 
75  3.20e+02   3.0e+02 1.4e-19 
76  3.20e+02   3.0e+02 1.4e-19 
77  3.20e+02   3.0e+02 1.4e-19 
78  3.20e+02   3.0e+02 1.4e-19 
79  3.20e+02   3.0e+02 1.4e-19 
80  3.20e+02   3.0e+02 1.4e-19 
81  3.20e+02   3.0e+02 1.4e-19 
82  3.20e+02   3.0e+02 1.4e-19 
83  3.20e+02   3.0e+02 1.4e-19 
84  3.20e+02   3.0e+02 1.4e-19 


85  3.20e+02   3.0e+02 1.4e-19 
86  3.20e+02   3.0e+02 1.4e-19 
87  3.20e+02   3.0e+02 1.4e-19 
88  3.20e+02   3.0e+02 1.4e-19 
89  3.20e+02   3.0e+02 1.4e-19 
90  3.20e+02   3.0e+02 1.4e-19 
91  3.20e+02   3.0e+02 1.4e-19 
92  3.20e+02   3.0e+02 1.4e-19 


93  3.20e+02   3.0e+02 1.4e-19 
94  3.20e+02   3.0e+02 1.4e-19 
95  3.20e+02   3.0e+02 1.4e-19 
96  3.20e+02   3.0e+02 1.4e-19 
97  3.20e+02   3.0e+02 1.4e-19 
98  3.20e+02   3.0e+02 1.4e-19 
99  3.20e+02   3.0e+02 1.4e-19 
100  3.20e+02   3.0e+02 1.4e-19 


100-element Vector{Float64}:
 -0.5811952904154614
 -0.22486493048939393
  1.459123201408629
  2.0219602260239244
  0.009764341626723894
  0.16203611449170155
 -0.01951899687752455
  0.18218382303118
  0.08387330440574498
  0.16320004921847162
  ⋮
  0.08259567605585669
  0.26229043017906717
  0.04382247839749778
  1.3341246089809105
  1.7315132849610428
  1.1116660372845282
  1.2253211597414129
  0.8483062566290611
  0.7064072882068685

In [118]:
bfgs_quasi_newton_armijo(model2, x0)

 k        fk ||∇f(x)||
 0  1.35e+03   8.8e+02
 1  5.66e+07   5.8e+06 8.8e-02 
 2  1.55e+13   5.0e+10 1.0e+00 
 3  5.14e+18   6.7e+14 4.4e-01 
 4  6.84e+24   3.5e+19 2.3e-03 
 5  3.22e+32   1.9e+25 2.3e-03 
 6  3.24e+42   6.1e+32 4.0e-05 
 7  3.25e+57   1.3e+44 5.9e-05 
 8  2.94e+76   1.3e+58 1.8e-11 
 9 3.01e+111   3.7e+84 1.6e-15 
10 5.90e+168  2.8e+127 1.7e-27 
11       Inf       Inf 1.0e+00 
12       NaN       NaN 1.0e+00 


100-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN